<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/main/SearchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup



adding stop words
1.general stopwords
2.stop words that are relevent to the subject -

Function to remove stop wards from dictionary- (current text)


1.Create the Hash Table that counts the amount of stem words from the text
2.Stem the words

In [ ]:
from nltk.stem import PorterStemmer
def apply_stemming(index):
  stemmer = PorterStemmer()
  stemmed_index = {}
  for word, count in index.items():
    stemmed_word = stemmer.stem(word)
    if stemmed_word in stemmed_index:
      stemmed_index[stemmed_word] += count
    else:
      stemmed_index[stemmed_word] = count
  return stemmed_index


Function
1.change all words to lower case
2. use stemming- it allows to search for sentences and not only single words

1. index_words - all words from the url
2. remove stop words - remove the stop words
3. apply stemming - change the words to the stem form
4. search - finds the words that are relevent and return thier location in the text

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
from nltk.stem import PorterStemmer

class searchEngine:
  def __init__(self):
    """Initialize the search engine"""
    self.base_url = "https://mqtt.org"
    self.pages = []
    self.word_locations = defaultdict(list)  # word -> [(page_id, frequency), ...]
    self.stop_words = {
        'a', 'an', 'the', 'and', 'or', 'but', 'if', 'then', 'else', 'for', 'on', 'in', 'with',
        'is', 'are', 'was', 'were', 'be', 'being', 'been',
        'of', 'to', 'from', 'by', 'as', 'at', 'it', 'its', 'that', 'this', 'these', 'those',
        'we', 'you', 'he', 'she', 'they', 'i', 'me', 'my', 'your', 'our', 'their',
        'which', 'who', 'whom', 'what', 'when', 'where', 'why', 'how', 'can', 'will', 'should', 'could'
    }
    self.stemmer = PorterStemmer()
  def fetch_pages(self, urls):
    """Fetch pages from specific MQTT.org URLs"""
    try:
        for url in urls:
            full_url = f"{self.base_url}{url}"
            print(f"Fetching: {full_url}")
            response = requests.get(full_url)
            if response.status_code != 200:
                print(f"Failed to fetch {full_url}")
                continue

            soup = BeautifulSoup(response.content, 'html.parser')
            title = soup.title.get_text(strip=True) if soup.title else 'No Title'
            content = soup.get_text(separator=' ', strip=True)

            page_id = len(self.pages) + 1
            self.pages.append({
                'id': page_id,
                'title': title,
                'url': full_url,
                'content': content
            })
        print(f"Retrieved {len(self.pages)} pages.")
        return True

    except Exception as e:
        print(f"Error: {e}")
        return False




  def build_index(self):
    """Build a simple word location index"""
    self.word_locations.clear()

    # Process each page
    for page in self.pages:
        # Get all words from content
        words = re.findall(r'\w+', page['content'].lower())

        # Count word frequencies
        word_counts = defaultdict(int)
        for word in words:
            if word not in self.stop_words:
                word_counts[word] += 1

        # Stemming
        word_counts = self.apply_stemming(word_counts)

        # Add to index with page information
        for word, count in word_counts.items():
            self.word_locations[word].append((page['id'], count))

  def apply_stemming(self, word_counts):
    """Apply stemming to the word counts"""
    return {self.stemmer.stem(word): count for word, count in word_counts.items()}

  def search(self, query, num_results=5):
    """Search pages using simple word frequency ranking"""
    # Get query words and apply stemming
    query_words = [
        self.stemmer.stem(word.lower())
        for word in re.findall(r'\w+', query)
        if word.lower() not in self.stop_words
    ]

    if not query_words:
        return []

    # Calculate scores for each page
    page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

    for word in query_words:
        for page_id, freq in self.word_locations.get(word, []):
            page_scores[page_id]['matches'] += 1
            page_scores[page_id]['total_freq'] += freq

    # Sort by the number of matches, then by total frequency
    ranked_results = [
        (page_id, scores['matches'], scores['total_freq'])
        for page_id, scores in page_scores.items()
    ]
    ranked_results.sort(key=lambda x: (x[1], x[2]), reverse=True)

    # Format results
    results = []
    for page_id, matches, total_freq in ranked_results[:num_results]:
        page = next(p for p in self.pages if p['id'] == page_id)
        original_query_words = [
            word.lower()
            for word in re.findall(r'\w+', query)
            if word.lower() not in self.stop_words
        ]
        context = self.get_context(page['content'], original_query_words)
        results.append({
            'title': page['title'],
            'url': page['url'],
            'matching_words': matches,
            'total_frequency': total_freq,
            'context': context
        })
    return results

  def get_context(self, content, query_words, window=30):
    """Get a short snippet of text around the first matching word"""
    content_lower = content.lower()
    for word in query_words:
        index = content_lower.find(word)
        if index != -1:
            start = max(index - window, 0)
            end = min(index + window, len(content))
            return content[start:end].strip()
    return "No context found."


In [ ]:
engine = searchEngine()

urls = [
    '/getting-started/',
    '/mqtt-specification/',
    '/use-cases/',
    '/software/',
    '/faq/',
    ''
]

engine.fetch_pages(urls)
engine.build_index()

results = engine.search("publish subscribe model")

for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Matching Words: {result['matching_words']}")
    print(f"Total Frequency: {result['total_frequency']}")
    print(f"Context: {result['context']}")
    print("-" * 60)

Fetching: https://mqtt.org/getting-started/
Fetching: https://mqtt.org/mqtt-specification/
Fetching: https://mqtt.org/use-cases/
Fetching: https://mqtt.org/software/
Fetching: https://mqtt.org/faq/
Fetching: https://mqtt.org
Retrieved 6 pages.
Result 1:
Title: Software
URL: https://mqtt.org/software/
Matching Words: 3
Total Frequency: 3
Context: TT clients while allowing the publishers to reliably transmi
------------------------------------------------------------
Result 2:
Title: MQTT - The Standard for IoT Messaging
URL: https://mqtt.org
Matching Words: 2
Total Frequency: 4
Context: d as an extremely lightweight publish/subscribe messaging tr
------------------------------------------------------------
Result 3:
Title: MQTT Specification
URL: https://mqtt.org/mqtt-specification/
Matching Words: 2
Total Frequency: 2
Context: such as Zigbee. MQTT-SN is a publish/subscribe messaging pr
------------------------------------------------------------
Result 4:
Title: FAQ
URL: https://mqtt.o